In [2]:
import pandas as pd
import numpy as np

In [3]:
orgData = pd.read_csv('/content/drive/MyDrive/Mini Project/Test_Signal_Data.csv')
orgData = orgData.iloc[:,1:5]
orgData['Error'] = 0
orgData

,Fc,Tc,Tr,Tj,Error
0,0.500,32.111391,35.444293,31.247891,0
1,0.500,32.263773,34.946917,31.552655,0
2,0.500,32.136788,35.413207,31.603449,0
3,0.500,32.416155,35.351035,31.933611,0
4,0.500,32.111391,35.413207,31.578052,0
...,...,...,...,...,...
10295,0.125,31.882817,59.473766,40.543212,0
10296,0.125,31.527258,59.411594,40.847977,0
10297,0.125,31.552655,59.411594,40.746389,0
10298,0.125,31.425670,59.038562,40.720992,0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
dataFc = pd.read_csv('/content/drive/MyDrive/Mini Project/FinalData/Fc_Error.xlsx')
dataTc = pd.read_csv('/content/drive/MyDrive/Mini Project/FinalData/Tc_error.csv')
dataTr = pd.read_csv('/content/drive/MyDrive/Mini Project/FinalData/Tr_error.csv')
dataTj = pd.read_csv('/content/drive/MyDrive/Mini Project/FinalData/Tj_error.csv')

dataTc = dataTc.iloc[:,2:]
dataTr = dataTr.iloc[:,2:]
dataTj = dataTj.iloc[:,2:]

data = pd.concat([dataFc,dataTc,dataTr,dataTj],ignore_index=True)
# data.to_csv('/content/drive/MyDrive/Mini Project/Fault_Present_Data.xlsx',index=False)

In [6]:
data[data["Error"] == 4]

,Fc,Tc,Tr,Tj,Error
92870,0.500,32.111391,35.972755,0.000000,4
92871,0.500,32.263773,36.221443,0.000000,4
92872,0.500,32.085993,35.848411,0.000000,4
92873,0.500,32.212979,35.910583,0.000000,4
92874,0.500,32.136788,36.034927,0.000000,4
...,...,...,...,...,...
123093,0.125,31.400273,60.499604,43.146687,4
123094,0.125,31.146303,61.214582,43.075980,4
123095,0.125,31.730435,60.375260,42.675112,4
123096,0.125,31.654244,60.686120,42.934566,4


In [7]:
X = data.iloc[:,[0,1,2,3]]
y = data.iloc[:,4]

# from imblearn.under_sampling import RandomUnderSampler
# rus = RandomUnderSampler(sampling_strategy='majority')
# X,y = rus.fit_resample(X,y)
# print("rev index:",X.shape)



from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train[['Fc','Tc','Tr','Tj']] = sc.fit_transform(X_train[['Fc','Tc','Tr','Tj']])
X_test[['Fc','Tc','Tr','Tj']] = sc.transform(X_test[['Fc','Tc','Tr','Tj']])

In [9]:
from sklearn.svm import SVC
from sklearn import metrics as met

In [10]:
import joblib
model = joblib.load("/content/drive/MyDrive/Mini Project/F_Model_45_Kernel_rbf_C_0.5_gamma_1_decision_ovr.pkl")
model.fit(X_train,y_train)
y_pred = model.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator SVC from version 1.2.0 when using version 1.2.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [11]:
coeff = model.dual_coef_

In [ ]:
model.support_vectors_

array([[ 0.45266759,  0.2047648 ,  0.17575015,  0.13797188],
       [ 0.17698684,  0.16129729, -0.94005019, -0.70245138],
       [ 1.05005683,  0.28735308,  0.42570542,  0.3250097 ],
       ...,
       [ 0.52565214,  0.03089475,  0.11776052, -0.37294868],
       [-0.51585056,  0.18303105,  0.47569647,  0.25549839],
       [-0.25954816,  0.05697526,  0.27973154,  0.52250226]])

In [ ]:
model.class_weight_

array([10., 10., 10., 10., 10.])

In [15]:
for i, class_coefficients in enumerate(coeff):
    print(f"Class {i+1}: {class_coefficients}")
    print(f'Lagrange multipliers of support vectors beloging to Class {i} : {len(class_coefficients)}')
    print("\n")

Class 1: [ 5.  0.  0. ... -0. -5. -5.]
Lagrange multipliers of support vectors beloging to Class 0 : 8900


Class 2: [ 0.          0.          0.         ... -0.         -1.00783652
 -0.        ]
Lagrange multipliers of support vectors beloging to Class 1 : 8900


Class 3: [ 0.  5.  0. ... -0. -0. -0.]
Lagrange multipliers of support vectors beloging to Class 2 : 8900


Class 4: [ 0.  0.  5. ... -5. -5. -5.]
Lagrange multipliers of support vectors beloging to Class 3 : 8900




In [ ]:
class_labels = model.classes_
print("Class Labels:", class_labels)

Class Labels: [0 1 2 3 4]


In [ ]:
from sklearn.model_selection import GridSearchCV

parameters_dictionary = {
                         'kernel':['rbf'],
                         'C':[0.1,1,5],
                         'gamma':[0.1,0.5],
                         'decision_function_shape':['ovr'],
                         'degree':[2]
                         }

svc = SVC(class_weight={0:10,1:10,2:10,3:10,4:10})
grid_search = GridSearchCV(svc,
                           parameters_dictionary,
                           cv = 3,
                           scoring = 'accuracy',
                           verbose = 4,
                          )
grid_search.fit(X_train, y_train)

Fitting 3 folds for each of 6 candidates, totalling 18 fits
[CV 1/3] END C=0.1, decision_function_shape=ovr, degree=2, gamma=0.1, kernel=rbf;, score=0.730 total time=  31.5s
[CV 2/3] END C=0.1, decision_function_shape=ovr, degree=2, gamma=0.1, kernel=rbf;, score=0.733 total time=  30.5s
[CV 3/3] END C=0.1, decision_function_shape=ovr, degree=2, gamma=0.1, kernel=rbf;, score=0.730 total time=  30.1s
[CV 1/3] END C=0.1, decision_function_shape=ovr, degree=2, gamma=0.5, kernel=rbf;, score=0.862 total time=  17.3s
[CV 2/3] END C=0.1, decision_function_shape=ovr, degree=2, gamma=0.5, kernel=rbf;, score=0.870 total time=  20.0s
[CV 3/3] END C=0.1, decision_function_shape=ovr, degree=2, gamma=0.5, kernel=rbf;, score=0.870 total time=  18.3s
[CV 1/3] END C=1, decision_function_shape=ovr, degree=2, gamma=0.1, kernel=rbf;, score=0.813 total time=  25.7s
[CV 2/3] END C=1, decision_function_shape=ovr, degree=2, gamma=0.1, kernel=rbf;, score=0.814 total time=  23.3s
[CV 3/3] END C=1, decision_funct

GridSearchCV(cv=3,
             estimator=SVC(class_weight={0: 10, 1: 10, 2: 10, 3: 10, 4: 10}),
             param_grid={'C': [0.1, 1, 5], 'decision_function_shape': ['ovr'],
                         'degree': [2], 'gamma': [0.1, 0.5],
                         'kernel': ['rbf']},
             scoring='accuracy', verbose=4)

In [ ]:
grid_search.best_params_

{'C': 5,
 'decision_function_shape': 'ovr',
 'degree': 2,
 'gamma': 0.5,
 'kernel': 'rbf'}

In [ ]:
grid_search.best_score_

0.9376008100274204

In [ ]:
grid_search.cv_results_['mean_test_score']

array([0.73102167, 0.86738795, 0.81413717, 0.91941495, 0.86246602,
       0.93760081])

In [ ]:
results = grid_search.cv_results_

C_values = [params['C'] for params in results['params']]
kernel_values = [params['kernel'] for params in results['params']]
gamma_values = [params['gamma'] for params in results['params']]
dec_values = [params['decision_function_shape'] for params in results['params']]
degree_values = [params['degree'] for params in results['params']]
scores = results['mean_test_score']

data = {
    'C': C_values,
    'Kernel': kernel_values,
    'gamma': gamma_values,
    'Degree':degree_values,
    'Decsion-Function':dec_values,
    'Score': scores,
}

parameterTuningData = pd.DataFrame(data)
parameterTuningData

,C,Kernel,gamma,Degree,Decsion-Function,Score
0,0.1,rbf,0.1,2,ovr,0.731022
1,0.1,rbf,0.5,2,ovr,0.867388
2,1.0,rbf,0.1,2,ovr,0.814137
3,1.0,rbf,0.5,2,ovr,0.919415
4,5.0,rbf,0.1,2,ovr,0.862466
5,5.0,rbf,0.5,2,ovr,0.937601


In [ ]:
import joblib
from sklearn.model_selection import ParameterGrid
from shutil import copyfile
import time

parameters_dictionary = {
                         'kernel':['rbf'],
                         'C':[0.1],
                         'gamma':[0.1],
                         'decision_function_shape':['ovr'],
                         'degree':[2]
                         }

i = 0
for params in ParameterGrid(parameters_dictionary):
    svc = SVC(class_weight={0: 10, 1: 10, 2: 10, 3: 10, 4: 10}, **params)
    svc.fit(X_train, y_train)
    filename = f'Model:::{time.strftime("%H:%M:%S")}||Kernel:{params["kernel"]}_C:{params["C"]}_gamma:{params["gamma"]}_decision:{params["decision_function_shape"]}_degree:{params["degree"]}.pkl'
    joblib.dump(svc, filename)
    source_path = filename
    target_path = '/content/drive/MyDrive/Mini Project/Model_Classifier_Pickles/'+f'{filename}'
    copyfile(source_path, target_path)
    print(f"Model saved for combination: {params}")
    del svc
    i = i+1

In [ ]:
new_Params = grid_search.best_params_
svc = SVC(class_weight={0: 10, 1: 10, 2: 10, 3: 10, 4: 10}, **new_Params)
svc.fit(X_train,y_train)
y_pred = svc.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score, classification_report,cohen_kappa_score,matthews_corrcoef,average_precision_score,log_loss,balanced_accuracy_score

# Assuming you have predicted labels 'y_pred' and true labels 'y_test'

# Accuracy
accuracy = accuracy_score(y_test, y_pred)

# Confusion Matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Precision
precision = precision_score(y_test, y_pred, average='macro')

# Recall
recall = recall_score(y_test, y_pred, average='macro')

# F1-Score
f1 = f1_score(y_test, y_pred, average='macro')

# ROC AUC Score
# roc_auc = roc_auc_score(y_test, y_pred)

# Classification Report
classification_rep = classification_report(y_test, y_pred)

# Cohen's Kappa measures the agreement between the predicted and true labels, taking into account the possibility of the agreement occurring by chance.
kappa = cohen_kappa_score(y_test, y_pred)

# MCC is a correlation coefficient between the predicted and true labels, considering all four quadrants of the confusion matrix.
mcc = matthews_corrcoef(y_test, y_pred)

# Average Precision Score calculates the average precision-recall curve for binary classification tasks.
# avg_precision = average_precision_score(y_test, y_pred,average='macro')


# Balanced Accuracy calculates the average accuracy of each class, taking into account class imbalance.
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)

In [ ]:
print("Accuracy",accuracy*100)
print("Precision",precision*100)
print("Recall",recall*100)
print("F1",f1*100)
print("Kappa",kappa*100)
print("MCC",mcc*100)
print("Balanced_accuracy",balanced_accuracy*100)
print("\n")
print("Classification report \n",classification_rep)

Accuracy 94.15665665665665
Precision 94.39273797703027
Recall 94.1648971561613
F1 94.18111319848167
Kappa 92.69541238999068
MCC 92.74441486101715
Balanced_accuracy 94.1648971561613


Classification report 
               precision    recall  f1-score   support

           0       0.88      0.97      0.92      4809
           1       0.97      0.96      0.97      4756
           2       1.00      0.99      0.99      4780
           3       0.90      0.91      0.90      4854
           4       0.97      0.88      0.92      4777

    accuracy                           0.94     23976
   macro avg       0.94      0.94      0.94     23976
weighted avg       0.94      0.94      0.94     23976



In [ ]:
# parameters_dictionary = {
#                          'kernel':['rbf', 'poly', 'linear','sigmoid'],
#                          'C':[0.1,0.2,0.5,1,10,100],
#                          'gamma':[0.1,1,10,100,1000,'scale','auto'],
#                          'decision_function_shape':['ovo','ovr'],
#                          'degree':[2,3]
#                          }